# Func

In [2]:
import os
from PIL import Image
import csv
from shutil import copyfile
# Function to convert RGB values to 16-bit 5-6-5 format
def rgb_to_565(r, g, b):
    r = (r >> 3) & 0x1F  # Red is 5 bits
    g = (g >> 2) & 0x3F  # Green is 6 bits
    b = (b >> 3) & 0x1F  # Blue is 5 bits
    return (r << 11) | (g << 5) | b  # Combine the RGB values into a 16-bit color

def color_to_char(color): # deprecated
    MSB = (color >> 8) & 0xFF
    LSB = color & 0xFF 
    return chr(MSB), chr(LSB), MSB, LSB

def color_to_raws(color):
    MSB = (color >> 8) & 0xFF
    LSB = color & 0xFF 
    return MSB,LSB

# Read the folder, process each jpg image
def process_images(folder_path):
    frames_raw = []
    frames_hex4 = []
    frames_int = []
    width = height = None
    
    # Read all JPG files in the directory
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            img = Image.open(image_path)
            img = img.convert("RGB")  # Ensure the image is in RGB format
            
            # Get width and height from the first image (assuming all frames have the same size)
            if width is None or height is None:
                width, height = img.size
            
            # Create a list to hold pixel colors for this frame
            frame_data_raw = []
            frame_data_hex4 = []
            frame_data_int = []
            # Iterate over each pixel in the image
            for y in range(height):
                for x in range(width):
                    r, g, b = img.getpixel((x, y))
                    color_code = rgb_to_565(r, g, b)
                    
                    MSB,LSB = color_to_raws(color_code);frame_data_raw.append(MSB);frame_data_raw.append(LSB);
                    frame_data_hex4.append(f"{color_code:04X}")
                    frame_data_int.append(color_code)
            
            # Add this frame's data to the frames list
            frames_raw.append(frame_data_raw)
            frames_hex4.append(frame_data_hex4)
            frames_int.append(frame_data_int)
            
    return frames_raw,frames_hex4, frames_int, width, height, len(frames_hex4)

# double file
def process_images_double(folder_path):
    frames_hex_0 = []
    frames_hex_1 = []
    width = height = None
    
    # Read all JPG files in the directory
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            img = Image.open(image_path)
            img = img.convert("RGB")  # Ensure the image is in RGB format
            
            # Get width and height from the first image (assuming all frames have the same size)
            if width is None or height is None:
                width, height = img.size
            
            # Create a list to hold pixel colors for this frame
            frame_data_hex_0 = []
            frame_data_hex_1 = []
            # Iterate over each pixel in the image
            for y in range(height//2):
                for x in range(width):
                    r, g, b = img.getpixel((x, y))
                    color_code = rgb_to_565(r, g, b)
                    frame_data_hex_0.append(f"{color_code:04X}")

            for y in range(height//2,height):
                for x in range(width):
                    r, g, b = img.getpixel((x, y))
                    color_code = rgb_to_565(r, g, b)
                    frame_data_hex_1.append(f"{color_code:04X}")
            
            # Add this frame's data to the frames list
            frames_hex_0.append(frame_data_hex_0)
            frames_hex_1.append(frame_data_hex_1)
            
    return frames_hex_0, frames_hex_1, width, height, len(frames_hex_0)
    
def stepFrame(folder_path,stepNum:int):
    output_folder_path = folder_path+"Step"+str(stepNum)
    if(os.path.exists(output_folder_path)):
        print("already generated!")
        return
    else:
        os.mkdir(output_folder_path)
        image_files = sorted(os.listdir(folder_path))
        for index, file in enumerate(image_files):
            if index % stepNum == 0:  # Every 3rd frame (0, 3, 6, ...)
                src = os.path.join(folder_path, file).replace("\\", "/")
                # print(src)
                dst = os.path.join(output_folder_path, file).replace("\\", "/")
                # print(dst)
                copyfile(src, dst)

# Function to write the data to a C header file
def write_header_file(frames, width, height, num_frames, output_file):
    total = width*height
    with open(output_file, "w") as f:
        f.write(f"int frames = {num_frames};\n")
        f.write(f"int animation_width = {width};\n")
        f.write(f"int animation_height = {height};\n")
        f.write("const unsigned short PROGMEM frame[]["+str(total)+"] = {\n")
        
        # Write each frame's data
        for frame in frames:
            f.write("    {")
            f.write(', '.join(map(str, frame)))
            f.write("},\n")
        
        f.write("};\n")
        
def write_txt_file(frames, output_file, scheme="utf-8"):
    with open(output_file, "w", encoding=scheme) as f:       
        # Write each frame's data
        for frame in frames:
            f.write(''.join(map(str, frame)))
            # f.write(' '.join(map(str, frame)))
            # f.write("\n")

def write_bin_file(frames, output_file):
    with open(output_file, "wb") as f:     
        # Write each frame's data
        for frame in frames:
            f.write(bytes(frame))
 

In [51]:
# test ASCII zipping
MSB_chr,LSB_chr,MSB,LSB = color_to_char(65530)
print(MSB_chr,LSB_chr,MSB,LSB)

output_txt_file = os.path.join(parent_folder,"testASCII.bin")
print(output_txt_file)
# with open(output_txt_file, "w",encoding="latin-1") as f:       
#     # Write each frame's data
#     f.write(MSB_chr)
#     f.write(LSB_chr)
with open(output_txt_file, "wb") as file:
    file.write(bytes([i for i in range(256)]))

ÿ ú 255 250
D:\_____ENGINEERING_____\arduino\AnimationTFT-main\myConverter\gifs/bluelain\testASCII.bin


# File

In [6]:
name = "bad_apple"
current_path = os.getcwd()

parent_folder = os.path.join(current_path, "gifs/"+name)
folder_path = os.path.join(parent_folder, name+"Frames").replace("\\", "/")
print("Folder path:",folder_path)

Folder path: D:/_____ENGINEERING_____/arduino/AnimationTFT-main/myConverter/gifs/bad_apple/bad_appleFrames


In [61]:
# run if need to step
step = 1
stepFrame(folder_path,step)
folder_path = os.path.join(parent_folder, name+"FramesStep"+str(step)).replace("\\", "/")
print("Folder path:",folder_path)

Folder path: D:/_____ENGINEERING_____/arduino/AnimationTFT-main/myConverter/gifs/bad_apple/bad_appleFramesStep1


In [7]:
# output_header_file = os.path.join(parent_folder,name+".h")
# output_txt_file_0 = os.path.join(parent_folder,name+"0.txt")
# output_txt_file_1 = os.path.join(parent_folder,name+"1.txt")
# output_txt_file = os.path.join(parent_folder,name+"ASCII1.txt")
output_bin_file = os.path.join(parent_folder,name+"ASCII.bin")

In [8]:
# Process the images in the folder
frames_raw, frames_hex4, frames_int, width, height, num_frames = process_images(folder_path)
# frames_hex_0, frames_hex_1, width, height, num_frames = process_images_double(folder_path)

# Write the results to the C header file
# write_header_file(frames_hex, width, height, num_frames, output_header_file)

# write_txt_file(frames_int, output_txt_file) # dont use unless test int
# write_txt_file(frames_hex_0, output_txt_file_0)
# write_txt_file(frames_hex_1, output_txt_file_1)
# write_txt_file(frames_char, output_txt_file,"latin-1")

write_bin_file(frames_raw, output_bin_file)
print(f"Files generated successfully.")

Files generated successfully.
